# V9

Well, Some novels, even with Chinese text, can still be difficult for me to read.
As a result, I think I have to ask LLM to rewrite it first.

## Gemini Pro API and OpenAI API
Check [Bilingual-NovelTranslator-v1.ipynb](Bilingual-NovelTranslator-v1.ipynb) for more information.

**Note** : Remember to fill you api key in `.env` file

In [1]:
%load_ext autoreload
%autoreload 2
    
import utils

geminipro, openai = utils.init_llm()

In [2]:

import utils
# refer to V6 for more info
utils.monkeypatch_split_text_with_regex()

# refer to V5 for more info
split_paragraph = utils.split_paragraph


In [3]:
def easy_version_paragraph(llm, paragraphs, target_file, start=0):
    from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
    from langchain_core.output_parsers.string import StrOutputParser
    prompt = PromptTemplate.from_template(
"""You are a professional novel writer. Your task is to rewrite certain passages of a novel into a much easier version for non-English adult readers.

Do not skip the chapter title if you found one

The passages came from ORC with a lots or type.
You should fix the typos and other mistakes, but you can just skip a sentence full of nonsense garbled characters.

In your output, every sentence or dialog should be separated and distinct.
Please make sure there is a line break after every complete sentence.

Just output the rewritten content, do not output any thing else.

----
The original passages is as following:

{input}

""")
    print(f"using llm: {llm}")
    chain = prompt | llm | StrOutputParser()
    revision=[]
    from tqdm.notebook import tqdm, trange
    errorindx = -1
    with tqdm(total=len(paragraphs)) as progress_bar:
        for i, paragraph in enumerate(paragraphs):
            if i < start:
                progress_bar.update(1)
                continue
            if len(paragraph.strip()) > 0:
                try:
                    # temp = ""
                    # for chunk in chain.stream({"input": paragraph}):
                    #     print(chunk, end="", flush=True)
                    #     temp += chunk
                    # print("\n------\n")    
                    temp = chain.invoke({"input": paragraph})
                except Exception as e:
                    print(f"Error occurs at {i}:")
                    print(paragraph)
                    print(e)
                    errorindx = i
                    return errorindx,revision;
 
                revision.append(temp)
                with open(target_file, 'a+') as f:
                    f.write(temp)
                    # f.write('\n')
            progress_bar.update(1)
    return errorindx,revision;

In [4]:
source_filenames = ["alice_in_wonderland.txt"]

In [5]:
errorbookindex=-1
errorindx=-1

In [6]:
for i, source_file in enumerate(source_filenames):
    if i < errorbookindex:
        continue

    print(source_file)
    target = source_file + ".v9.txt"
    path = source_file
    paragraphs = split_paragraph(path)[1:5]

    progress=0
    while (progress < len(paragraphs)):
        errorindx, translated_paragraphs = easy_version_paragraph(geminipro, paragraphs, target, start=errorindx)        
        if errorindx < 0:
            progress=len(paragraphs)
        else: # errorindx >= 0:# error, retry
            _errorindx = -1
            for _llm in [openai]: # use optional llm to retry
                _paragraphs = paragraphs[errorindx:errorindx+1]
                _errorindx, _translated_paragraphs = easy_version_paragraph(_llm, _paragraphs, target, start=0)
                if _errorindx < 0: # retry success
                    break
            if _errorindx < 0: # any retry success
                errorindx=errorindx+1 # continue next
                progress = errorindx
                continue

    if errorindx >= 0:
        errorbookindex=i
        break

alice_in_wonderland.txt
using llm: model='gemini-pro' temperature=0.5 top_p=0.85 safety_settings={<HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>: <HarmBlockThreshold.BLOCK_NONE: 4>} client= genai.GenerativeModel(
   model_name='models/gemini-pro',
   generation_config={}.
   safety_settings={}
) convert_system_message_to_human=True


  0%|          | 0/4 [00:00<?, ?it/s]

### Now we can make another bilangual audio book.

In [7]:
# refer to V7 for more info
autoprocess_paragraph = utils.autoprocess_paragraph

In [8]:
source_filenames = ["alice_in_wonderland.txt.v9.txt"]

In [9]:
errorbookindex=-1
errorindx=-1

In [10]:
for i, source_file in enumerate(source_filenames):
    if i < errorbookindex:
        continue

    print(source_file)
    target = source_file + ".json.txt"
    path = source_file
    paragraphs = split_paragraph(path)

    progress=0
    while (progress < len(paragraphs)):
        errorindx, translated_paragraphs = autoprocess_paragraph(geminipro, paragraphs, target, start=errorindx)        
        if errorindx < 0:
            progress=len(paragraphs)
        else: # errorindx >= 0:# error, retry
            _errorindx = -1
            for _llm in [openai]: # use optional llm to retry
                _paragraphs = paragraphs[errorindx:errorindx+1]
                _errorindx, _translated_paragraphs = autoprocess_paragraph(_llm, _paragraphs, target, start=0)
                if _errorindx < 0: # retry success
                    break
            if _errorindx < 0: # any retry success
                errorindx=errorindx+1 # continue next
                progress = errorindx
                continue

    if errorindx >= 0:
        errorbookindex=i
        break

alice_in_wonderland.txt.v9.txt
using llm: model='gemini-pro' temperature=0.5 top_p=0.85 safety_settings={<HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>: <HarmBlockThreshold.BLOCK_NONE: 4>} client= genai.GenerativeModel(
   model_name='models/gemini-pro',
   generation_config={}.
   safety_settings={}
) convert_system_message_to_human=True


  0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
import re
import json


with open("alice_in_wonderland.txt.v9.txt.json.txt", "r") as file:
    content = file.read()

pattern = re.compile(r"```json(.*?)```", re.DOTALL)
matches = pattern.findall(content)

merged_list = []

# 解析每個區塊並將其加入到合併後的list中
for match in matches:
    try:
        from json_repair import repair_json
        json_data = repair_json(match.strip(), return_objects=True)
        merged_list.extend(json_data)
    except json.JSONDecodeError as e:
        print(e)
        print(f"Error decoding JSON in block: {match}")

with open("alice_in_wonderland.txt.v9.zhtw.txt", "w+") as f:
    for ele in merged_list:
        f.write("" + ele["zh-tw"] + "\n")
        f.write("" + ele["correction"] + "\n\n")

Enjoy your new novel in `alice_in_wonderland.txt.v9.zhtw.txt`